## Setup environment

In [1]:

from monai.utils import first, set_determinism
import torch
import matplotlib.pyplot as plt
import os
import torch.nn as nn
from data_preparation import DataHandling 
from datetime import datetime
import json
import numpy as np


## Set dataset path

In [2]:
config_file = 'config.json'

with open(config_file, 'r') as f:
    config = json.load(f)

ga_data_dir = config["ga_data_dir"]
fdg_data_dir = config["fdg_data_dir"]
log_dir = config["log_dir"]
output_dir = config["output_dir"]


In [3]:
import os
import glob
from utils import PairFinder
# hint = 'dl_dyn3'
# hint = 'gamodel_3_18_onfdg'
# hint = 'gamodel_onfdg'
# hint = 'test_corr2'
# hint = 'comb_3_27_11_26_onfdg'
# hint = 'comb_3_27_11_26_onga'
#hint = 'comb_3_27_11_26_onfdg_spacing'
hint = 'dl_dyn1'
pair_finder = PairFinder(f'{ga_data_dir}/MAC', output_dir, hint)
pair_finder = PairFinder(f'{ga_data_dir}/MAC', output_dir, hint)
all_pairs, c5_pairs, rest_pairs = pair_finder.find_file_pairs()


In [4]:
# import nibabel as nib
# import numpy as np
# import matplotlib.pyplot as plt

# def load_nifti_image(path):
#     """Load a NIfTI image and return its data as a NumPy array."""
#     return nib.load(path).get_fdata()

# def create_mask_from_reference(reference_img, mask_threshold=0.03, scaling_factor=5):
#     """Create a mask from the reference image based on a threshold."""
#     scaled_reference_img = reference_img * scaling_factor
#     mask = scaled_reference_img > mask_threshold
#     return mask

# def apply_mask(image, mask, scaling_factor=5):
#     """Apply the mask to the image."""
#     scaled_image = image * scaling_factor
#     masked_image = np.zeros_like(scaled_image)
#     masked_image[mask] = scaled_image[mask]
#     return masked_image


# def plot_coronal_slice(img, title="Image slice", cmap="jet", slice_idx=None):
#     """Plots a coronal slice of an image."""
#     if slice_idx is None:
#         slice_idx = 70  
#     plt.figure()
#     plt.imshow(img[:, slice_idx, :], cmap=cmap, origin='lower')
#     plt.title(title)
#     plt.axis("off")
#     plt.show()

# # Loop through each pair of images in the list
# for pair in test_dict_list:
#     predicted_img_path = pair['predicted']
#     reference_img_path = pair['reference']
    
#     # Load the images
#     predicted_img = load_nifti_image(predicted_img_path)
#     reference_img = load_nifti_image(reference_img_path)
    
#     # Create a mask from the reference image
#     mask = create_mask_from_reference(reference_img)
    
#     # Apply the same mask to both images
#     masked_predicted_img = apply_mask(predicted_img, mask)
#     masked_reference_img = apply_mask(reference_img, mask)
    
  
#     # # Plot a coronal slice for both masked images
#     # plot_coronal_slice(masked_predicted_img, title=f"Masked Predicted: {predicted_img_path.split('/')[-1]}")
#     # plot_coronal_slice(masked_reference_img, title=f"masked Reference: {reference_img_path.split('/')[-1]}")

#     # plot_coronal_slice(reference_img, title=f"unmasked Reference: {reference_img_path.split('/')[-1]}")

#     break
    

------------------
# Quantification Evaluation

In [4]:
import numpy as np
import nibabel as nib
from math import sqrt, log10
from skimage.metrics import structural_similarity as ssim

def mean_error(predicted, reference):
    return np.mean(predicted - reference)

def mean_absolute_error(predicted, reference):
    return np.mean(np.abs(predicted - reference))

def relative_error(predicted, reference):

    re = np.mean((predicted - reference) / (reference )) * 100
    return re

def absolute_relative_error(predicted, reference):
    # Create a mask for pixels in the reference image above the threshold

    # Calculate the absolute relative error using the masked pixels
    are = np.mean(np.abs(predicted - reference) / reference) * 100
    
    return are

def rmse(predicted, reference):
    return sqrt(np.mean((predicted - reference) ** 2))

def psnr(predicted, reference, peak):
    mse = np.mean((predicted - reference) ** 2)
    return 20 * log10(peak / sqrt(mse))

def calculate_ssim(predicted, reference):
    return ssim(predicted, reference, data_range=reference.max() - reference.min())

def load_nifti_image(path):
    """Load a NIfTI image and return its data as a NumPy array."""
    return nib.load(path).get_fdata()

def calculate_metrics_for_pair(predicted_path, reference_path, scaling_factor, mask_val):
    """
    Calculate metrics for a single pair of images, applying a scaling factor to the images.
    A mask is applied where the reference image values are bigger than 0.03.
    """
    predicted_img = load_nifti_image(predicted_path) * scaling_factor
    reference_img = load_nifti_image(reference_path) * scaling_factor
    # print(np.min(predicted_img), np.max(predicted_img))
    # print(np.min(reference_img), np.max(reference_img))
    # print("---------------------------------------------")
    # Create mask from reference image where values are greater than 0.03
    mask = reference_img > mask_val
    
    # Apply the mask to both images
    masked_predicted_img = predicted_img[mask]
    masked_reference_img = reference_img[mask]

    peak = np.max([masked_predicted_img.max(), masked_reference_img.max()])
    metrics = {
        "mean_error": mean_error(masked_predicted_img, masked_reference_img),
        "mean_absolute_error": mean_absolute_error(masked_predicted_img, masked_reference_img),
        "relative_error": relative_error(masked_predicted_img, masked_reference_img),
        "absolute_relative_error": absolute_relative_error(masked_predicted_img, masked_reference_img),
        "rmse": rmse(masked_predicted_img, masked_reference_img),
        "psnr": psnr(masked_predicted_img, masked_reference_img, peak),
        "ssim": calculate_ssim(masked_predicted_img, masked_reference_img)
    }
    return metrics

def aggregate_metrics(metrics_list):
    """Aggregate metrics across all pairs and calculate mean and standard deviation."""
    aggregated_metrics = {key: [] for key in metrics_list[0]}
    for metrics in metrics_list:
        for key, value in metrics.items():
            aggregated_metrics[key].append(value)
    
    return {metric: (np.mean(values), np.std(values)) for metric, values in aggregated_metrics.items()}

In [7]:
# from utils import calculate_metrics_for_pair, aggregate_metrics

# Calculate metrics for each pair and aggregate results
all_metrics = [calculate_metrics_for_pair(
    pair['predicted'], pair['reference'],
    scaling_factor=5, mask_val = 0.3)
    for pair in rest_pairs]

metric_means_sds = aggregate_metrics(all_metrics)

# Print aggregated metrics
for metric, (mean, sd) in metric_means_sds.items():
    print(f"{metric}: {mean:.2f} ± {sd:.4f}")


mean_error: -0.42 ± 0.7114
mean_absolute_error: 1.30 ± 0.3018
relative_error: 2.88 ± 16.0516
absolute_relative_error: 35.94 ± 5.5011
rmse: 3.48 ± 1.7411
psnr: 37.04 ± 1.8921
ssim: 0.94 ± 0.0325


In [ ]:
import numpy as np

# Example data
predicted = np.array([0.1, 0.2, 1.05, 0.4])
reference = np.array([0.15, 0.25, 0.02, 0.35])

# Threshold
threshold = 0.03

# Mask where reference values are greater than the threshold
mask = reference > threshold

# Applying the mask
masked_predicted = predicted[mask]
masked_reference = reference[mask]

# Original data becomes subset
print("Masked Predicted:", masked_predicted)
print("Masked Reference:", masked_reference)


Masked Predicted: [0.1 0.2 0.4]
Masked Reference: [0.15 0.25 0.35]


# Results for Dynunet without matching the networks prameters:
mean_error: -0.25 ± 0.0722
mean_absolute_error: 0.43 ± 0.0803
relative_error: 7.56 ± 7.7195
absolute_relative_error: 38.52 ± 4.3638
rmse: 2.10 ± 0.9815
psnr: 39.60 ± 3.0782
ssim: 0.97 ± 0.0040


# Results for Dyunet with matching the networks prameters:

mean_error: -0.27 ± 0.0802
mean_absolute_error: 0.37 ± 0.0524
relative_error: -14.82 ± 2.6155
absolute_relative_error: 35.17 ± 3.3576
rmse: 1.60 ± 0.3624
psnr: 41.57 ± 2.2030
ssim: 0.97 ± 0.0056


# Resulats for Dyunet after some augmentation:

mean_error: -0.13 ± 0.1053
mean_absolute_error: 0.29 ± 0.1097
relative_error: 11.28 ± 9.1598
absolute_relative_error: 24.58 ± 7.9870
rmse: 1.59 ± 0.9717
psnr: 43.46 ± 4.1015
ssim: 0.98 ± 0.0063

# Results for re-trained model with fdg data on fdg datatest
mean_error: 0.02 ± 0.0064
mean_absolute_error: 0.08 ± 0.0111
relative_error: 6.71 ± 0.0923
absolute_relative_error: 10.90 ± 0.1017
rmse: 0.30 ± 0.0762
psnr: 46.35 ± 6.6213
ssim: 0.98 ± 0.0056